<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84(%2B%EB%B0%B1%ED%85%8C%EC%8A%A4%ED%8A%B8)_RAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install yfinance

In [27]:
!pip install pandas_market_calendars

In [28]:
!pip install fredapi

In [29]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv
from fredapi import Fred

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
# start_date = today - timedelta(days=375)

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

# 전략명
str_strategy = 'RAA '

# 오늘 날짜를 가져와서 파일명에 추가
today_date = today.strftime('%y%m%d')
filename = f'{today_date} {str_strategy}.xlsx'

# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 월평균영업일수. average Monthly Business Day
MBD = 21

# 전략에 사용할 ETF 명 리스트에 담기
etf_stock = ['QQQ', 'IWN']
etf_bond = ['IEF', 'TLT']
etf_real = ['GLD']
etf_canary = ['VWO', 'BND']

# 자산군 리스트 더하기, 중복 제거
etf_order = list(set(etf_stock + etf_bond + etf_real + etf_canary))

# ETF 가격 추출
df_etf = yf.download(etf_order, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0).dropna()
display(df_etf)

# 1개월씩 자르기
df_etf_monthly = df_etf[::MBD]

[*********************100%%**********************]  7 of 7 completed


,IEF,BND,QQQ,TLT,GLD,IWN,VWO
2023-09-19,92.8400,70.7100,369.8700,92.8000,179.2200,139.5200,39.9500
2023-09-18,93.1800,70.8600,370.6600,93.4900,179.3900,139.9000,40.1500
2023-09-15,93.0900,70.7900,370.2740,92.9600,178.3400,141.0000,40.2120
2023-09-14,93.3800,70.9300,376.7247,93.5300,177.1700,142.3800,40.4203
2023-09-13,93.5900,71.0500,373.6691,94.2100,177.0800,139.9800,40.1822
...,...,...,...,...,...,...,...
2007-04-16,56.2565,45.5329,39.2398,54.2871,68.4000,60.5503,26.9891
2007-04-13,56.1952,45.4904,38.8827,53.9896,67.8400,59.8035,26.6553
2007-04-12,56.2907,45.5633,38.8043,54.1570,66.9900,59.4265,26.5686
2007-04-11,56.2361,45.5694,38.4995,54.1446,67.0800,59.0494,26.2348


In [30]:
# api key 불러오기
import json

api_key_data = ''

with open('/content/drive/My Drive/DynamicAssetAllocation/api_key.json') as f:
    api_key_data = json.load(f)


## DataFrame 만들기: 미국 실업률 (ex. 08-01: 8월1일~31일 실업률 통계)
fred = Fred(api_key=api_key_data['fred'])
fred_unrate = fred.get_series('UNRATE').sort_index(ascending=False)
df_fred_unrate = pd.DataFrame(fred_unrate, columns=['unemployment rate'])

str_fred_unrate = str_today + str_strategy + "### 미국 실업률"
display(Markdown(str_fred_unrate))
display(df_fred_unrate.head(5))

with open(output_path + filename, 'w') as f:
  f.write(f'{str_fred_unrate}\n')
  df_fred_unrate.head(5).to_csv(f, index=True)

### 2023-09-20 RAA ### 미국 실업률

,unemployment rate
2023-08-01,3.8000
2023-07-01,3.5000
2023-06-01,3.6000
2023-05-01,3.7000
2023-04-01,3.4000


In [31]:
# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)

### 2023-09-20 RAA ### ETF 현재-1-3-6-12 개월 가격

,IEF,BND,QQQ,TLT,GLD,IWN,VWO
today/2023-09-19,92.8400,70.7100,369.8700,92.8000,179.2200,139.5200,39.9500
1mo_ago/2023-08-18,93.3611,70.6764,357.6123,93.4896,175.3300,142.1300,39.4978
3mo_ago/2023-06-20,96.4349,72.2541,366.3697,102.4405,179.7500,139.6500,40.6385
6mo_ago/2023-03-20,97.4623,72.2771,305.1092,104.2124,183.8400,132.6673,38.2580
12mo_ago/2022-09-16,96.1029,71.0539,286.8566,103.8198,155.8400,138.1603,37.9661


In [32]:
## DataFrame 만들기: 모멘텀 스코어
df_etf_monthly_copy = df_etf_monthly.copy(deep=True)
df_etf_momentum_score = pd.DataFrame(index=df_etf_monthly.index, columns=df_etf_monthly.columns)

for i in range(0, len(df_etf_monthly) - 12):
  ror_1mo = df_etf_monthly_copy.pct_change(-1).iloc[0]  # 1개월 수익률
  ror_3mo = df_etf_monthly_copy.pct_change(-3).iloc[0]  # 3개월 수익률
  ror_6mo = df_etf_monthly_copy.pct_change(-6).iloc[0]  # 6개월 수익률
  ror_12mo = df_etf_monthly_copy.pct_change(-12).iloc[0]  # 12개월 수익률
  df_etf_monthly_copy.drop(df_etf_monthly_copy.index[0], inplace=True)

  # 가중치 더하기
  df_etf_momentum_score.iloc[i] = ((12 * ror_1mo) + (4 * ror_3mo) + (2 * ror_6mo) + (ror_12mo)) * 100

df_etf_momentum_score.dropna(inplace=True)

str_etf_momentum_score = str_today + str_strategy + "### ETF 별 1-3-6-12 개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.head(5).to_csv(f, index=True)

### 2023-09-20 RAA ### ETF 별 1-3-6-12 개월 Momentum Score

,IEF,BND,QQQ,TLT,GLD,IWN,VWO
2023-09-19,-34.4890,-12.7984,116.3432,-79.0111,35.4211,-11.0937,21.0311
2023-08-18,-56.0835,-37.9782,12.6310,-147.4099,-50.9342,-42.7766,-50.5931
2023-07-20,-19.7053,-5.5339,204.9162,-39.8910,35.2417,123.0583,27.9822
2023-06-20,-13.6066,3.0665,290.3707,2.3148,-1.1855,84.6218,78.9291
2023-05-18,14.2000,6.6266,160.3850,-19.6667,30.3255,-91.3272,-22.1670
...,...,...,...,...,...,...,...
2008-08-12,25.7790,-0.8310,104.8027,11.2177,-213.5719,196.1918,-153.8490
2008-07-14,47.9510,30.8677,-101.1058,69.8911,227.4084,-181.0357,-169.8085
2008-06-12,-25.8910,-23.0095,-16.5943,-33.8877,0.9311,-41.8686,-66.6659
2008-05-13,-12.7500,6.8580,193.0827,-34.4848,-56.3356,72.0778,132.0056


In [33]:
## DataFrame 만들기: ETF 종목, 종목별 투자 비율
row_count = len(df_etf_momentum_score)
df_high_etf = pd.DataFrame(columns=[0, 1, 2, 3, 4], index=df_etf_momentum_score.index)

df_buy_ratio_etf = pd.DataFrame(columns=[0, 1, 2, 3, 4], index=df_etf_momentum_score.index)
bond_portfolio_ratio = [0.5] *2 + [0] * 3
stock_portfolio_ratio = [0.2] * 5

# 마지막에 출력할 str 정의
str_fred_unrate = ''
str_canary_minus = ''
str_buy_etf = ''
str_etf_ratio = ''

for i in reversed(range(row_count)):
  # 지금 실업률이 1년전 실업률보다 높은가? (낮으면 5개 자산 투자. 주식 40%, 채권 40%, 실물자산 20%)
  if df_fred_unrate.iloc[i][0] > df_fred_unrate.iloc[i+12][0]:
    str_fred_unrate = 'True'

  # 높으면 VWO, BND 1-3-6-12 모멘텀스코어가 하나라도 마이너스인가?
    if df_etf_momentum_score.iloc[i]['VWO'] < 0 or df_etf_momentum_score.iloc[i]['BND'] < 0:
      str_canary_minus = 'True'

      # 하나라도 마이너스이면 채권 포트폴리오 투자 (IEF 50%, TLT 50%)
      df_high_etf.iloc[i] = [etf_bond[0]] + [etf_bond[1]] * 4
      df_buy_ratio_etf.iloc[i] = bond_portfolio_ratio

    # 둘 다 플러스이면 5개 자산 비율 투자
    else:
      df_high_etf.iloc[i] = etf_stock + etf_bond + etf_real
      df_buy_ratio_etf.iloc[i] = stock_portfolio_ratio

      str_canary_minus = 'False'

  # 지금 실업률이 1년전 실업률보다 낮음
  else:
    df_high_etf.iloc[i] = etf_stock + etf_bond + etf_real
    df_buy_ratio_etf.iloc[i] = stock_portfolio_ratio

    str_fred_unrate = 'False'
    str_canary_minus = ''

  # 최신 데이터로 종목 뽑기
  if i == 0:
    str_buy_etf = ' '.join(df_high_etf.iloc[i].drop_duplicates())
    str_etf_ratio = str(round(df_buy_ratio_etf[df_buy_ratio_etf > 0].iloc[i].drop_duplicates().dropna()[0] * 100)) + ' %'

str_high_etf = str_today + str_strategy + "### ETF 투자 종목 선정"
display(Markdown(str_high_etf))
display(df_high_etf)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_high_etf}\n')
  df_high_etf.head(5).to_csv(f, index=True)

### 2023-09-20 RAA ### ETF 투자 종목 선정

,0,1,2,3,4
2023-09-19,IEF,TLT,TLT,TLT,TLT
2023-08-18,QQQ,IWN,IEF,TLT,GLD
2023-07-20,QQQ,IWN,IEF,TLT,GLD
2023-06-20,QQQ,IWN,IEF,TLT,GLD
2023-05-18,QQQ,IWN,IEF,TLT,GLD
...,...,...,...,...,...
2008-08-12,IEF,TLT,TLT,TLT,TLT
2008-07-14,IEF,TLT,TLT,TLT,TLT
2008-06-12,IEF,TLT,TLT,TLT,TLT
2008-05-13,QQQ,IWN,IEF,TLT,GLD


In [34]:
# 종목별 투자 비율
df_buy_ratio = df_buy_ratio_etf

str_buy_ratio = str_today + str_strategy + "### ETF 투자 비율"
display(Markdown(str_buy_ratio))
display(df_buy_ratio)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_buy_ratio}\n')
  df_buy_ratio.head(5).to_csv(f, index=True)

### 2023-09-20 RAA ### ETF 투자 비율

,0,1,2,3,4
2023-09-19,0.5000,0.5000,0,0,0
2023-08-18,0.2000,0.2000,0.2000,0.2000,0.2000
2023-07-20,0.2000,0.2000,0.2000,0.2000,0.2000
2023-06-20,0.2000,0.2000,0.2000,0.2000,0.2000
2023-05-18,0.2000,0.2000,0.2000,0.2000,0.2000
...,...,...,...,...,...
2008-08-12,0.5000,0.5000,0,0,0
2008-07-14,0.5000,0.5000,0,0,0
2008-06-12,0.5000,0.5000,0,0,0
2008-05-13,0.2000,0.2000,0.2000,0.2000,0.2000


In [35]:
## 백테스트
monthly_column_count = df_high_etf.shape[1]
monthly_index = df_high_etf.index

# 비율대로 돈 투자
df_invest_money = pd.DataFrame(columns=[i for i in range(monthly_column_count)], index=monthly_index)
df_invest_count = pd.DataFrame(columns=[i for i in range(monthly_column_count)], index=monthly_index)
df_cash = pd.DataFrame(columns=['cash'], index=monthly_index)

# 초기자본
invest_cash = 10_000

for i in reversed(range(row_count)):
  row_sum = 0

  for j in range(monthly_column_count):
    if i == row_count - 1:
      row_sum = invest_cash
      break
    else:
      sel_etf = df_high_etf.iloc[i+1][j]
      row_sum += df_invest_count.iloc[i+1][j] * df_etf_monthly.iloc[i][sel_etf]

  for j in range(monthly_column_count):
    df_cash.iloc[i][0] = row_sum
    sel_etf = df_high_etf.iloc[i][j]
    sel_etf_price = df_etf_monthly.iloc[i][sel_etf]
    sel_etf_buy_count = df_cash.iloc[i][0] * df_buy_ratio.iloc[i][j] / sel_etf_price
    df_invest_count.iloc[i][j] = sel_etf_buy_count

display(Markdown(str_today + str_strategy + '### ETF 구매 개수'))
display(df_invest_count)
display(Markdown(str_today + str_strategy + '### 평가 금액'))
display(df_cash)

CAGR = str(round(((df_cash.iloc[0][0] / df_cash.iloc[-1][0]) ** (12/(row_count-1)) - 1)*100, 2))
display(Markdown(f'{str_today[1:]} {str_strategy} ## CAGR: {CAGR} %'))

### 2023-09-20 RAA ### ETF 구매 개수

,0,1,2,3,4
2023-09-19,181.1004,181.1785,0.0000,0.0000,0.0000
2023-08-18,18.7122,47.0815,71.6755,71.5769,38.1663
2023-07-20,18.6561,47.3406,73.2099,69.4116,38.4082
2023-06-20,18.8320,49.4056,71.5456,67.3512,38.3838
2023-05-18,19.9760,50.3145,69.2338,66.7194,36.9464
...,...,...,...,...,...
2008-08-12,77.0074,81.2796,0.0000,0.0000,0.0000
2008-07-14,77.4734,80.7877,0.0000,0.0000,0.0000
2008-06-12,76.8146,81.4747,0.0000,0.0000,0.0000
2008-05-13,45.0866,37.7195,30.7603,32.5505,22.7025


### 2023-09-20 RAA ### 평가 금액

,cash
2023-09-19,33626.7298
2023-08-18,33458.4838
2023-07-20,35093.5485
2023-06-20,34497.4489
2023-05-18,33591.6984
...,...
2008-08-12,9853.4366
2008-07-14,9864.4483
2008-06-12,9495.6715
2008-05-13,9692.8385


## 2023-09-20  RAA  ## CAGR: 8.18 %

In [36]:
## 결과 출력
display(Markdown(str_today[1:] + str_strategy))

str_result_etf = '## 지금 실업률이 1년전 실업률보다 높은가? (낮으면 5개 자산 투자. 주식 40%  채권 40%  실물자산 20%): ' + str_fred_unrate
display(Markdown(str_result_etf))

str_result_momentum = '## 높으면 VWO, BND 1-3-6-12 모멘텀스코어가 하나라도 마이너스인가?: ' + str_canary_minus
display(Markdown(str_result_momentum))

str_result_buy = '## 구매할 자산: ' + str_buy_etf
display(Markdown(str_result_buy))

str_result_ratio = '## 각각 구매 비율: ' + str_etf_ratio
display(Markdown(str_result_ratio))

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_result_etf}\n')
  f.write(f'{str_result_momentum}\n')
  f.write(f'{str_result_buy}\n')
  f.write(f'{str_result_ratio}\n')

## 2023-09-20 RAA 

## 지금 실업률이 1년전 실업률보다 높은가? (낮으면 5개 자산 투자. 주식 40%  채권 40%  실물자산 20%): True

## 높으면 VWO, BND 1-3-6-12 모멘텀스코어가 하나라도 마이너스인가?: True

## 구매할 자산: IEF TLT

## 각각 구매 비율: 50 %